<a href="https://colab.research.google.com/github/omkar-salunke/accounts/blob/main/both_tally_daybook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# prompt: list of files in the path in python

import os
import pandas as pd
import xml.etree.ElementTree as ET
import pandas as pd

# Function to recursively extract text and sub-elements from a node
def extract_element_data(element):
    element_data = {}
    # If the element has no children, return its text
    if len(element) == 0:
        return element.text

    # Otherwise, go deeper into the element's children
    for child in element:
        # Recursively extract data for each child element
        child_data = extract_element_data(child)
        element_data[child.tag] = child_data
    return element_data

# Function to flatten nested dictionaries (e.g., 'amount': {'P': {'T': 100}})
def flatten_dict(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Function to recursively search for all 'voucher' elements and extract their data
def find_all_vouchers(element, voucher_tag="VOUCHER"):
    vouchers = []

    # If the current element is a 'voucher', collect its data
    if element.tag == voucher_tag:
        voucher_dict = {}
        # Collect data for all child elements within the voucher
        for sub_child in element:
            extracted_data = extract_element_data(sub_child)
            voucher_dict[sub_child.tag] = extracted_data

        # Flatten the voucher dictionary to handle nested elements
        voucher_dict_flattened = flatten_dict(voucher_dict)
        vouchers.append(voucher_dict_flattened)

    # Recursively search in all children of the current element
    for child in element:
        vouchers.extend(find_all_vouchers(child, voucher_tag))

    return vouchers

# Function to convert the XML to a DataFrame by extracting all vouchers
def xml_to_dataframe_vouchers(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Find all 'voucher' elements in the XML
    voucher_list = find_all_vouchers(root)

    # Create a DataFrame from the list of voucher dictionaries
    df = pd.DataFrame(voucher_list)
    return df




In [ ]:
df_bank_match_soe = pd.read_excel("/content/drive/My Drive/2024_25/overview_account.xlsx",sheet_name='SOE 2024-25')
df_bank_match_overview_soe = df_bank_match_soe.groupby('comment').agg({'Debit':'sum','Credit':'sum'}).reset_index().rename(columns={'comment':'party_name'})

df_bank_match_me = pd.read_excel("/content/drive/My Drive/2024_25/overview_account.xlsx",sheet_name='ME 2024-25')
df_bank_match_overview_me = df_bank_match_me.groupby('comment').agg({'Debit':'sum','Credit':'sum'}).reset_index().rename(columns={'comment':'party_name'})

In [ ]:
files_soe = os.listdir('/content/drive/My Drive/2024_25/SOE/')
files_me = os.listdir('/content/drive/My Drive/2024_25/ME/')

In [ ]:

req_cols = ['DATE','GUID','VOUCHERTYPENAME','PARTYLEDGERNAME','PARTYGSTIN','CLASSNAME','PARTYNAME','CONSIGNEEGSTIN','PERSISTEDVIEW','ALLINVENTORYENTRIES.LIST.STOCKITEMNAME','ALLINVENTORYENTRIES.LIST.RATE','ALLINVENTORYENTRIES.LIST.AMOUNT','LEDGERENTRIES.LIST.VATEXPAMOUNT','LEDGERENTRIES.LIST.AMOUNT','ALLINVENTORYENTRIES.LIST.ACTUALQTY','ALLINVENTORYENTRIES.LIST.BILLEDQTY','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.ACTUALQTY','ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.BILLEDQTY','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.LEDGERNAME','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.AMOUNT','LEDGERENTRIES.LIST.LEDGERNAME','LEDGERENTRIES.LIST.AMOUNT','LEDGERENTRIES.LIST.VATEXPAMOUNT','INVOICEORDERLIST.LIST.BASICORDERDATE','INVOICEORDERLIST.LIST.BASICPURCHASEORDERNO','REFERENCEDATE','REFERENCE','CONSIGNEEPINNUMBER','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.GSTOVRDNNATURE','ALLINVENTORYENTRIES.LIST.ACCOUNTINGALLOCATIONS.LIST.RATEDETAILS.LIST.GSTRATEDUTYHEAD','ALLLEDGERENTRIES.LIST.OLDAUDITENTRYIDS.LIST.OLDAUDITENTRYIDS','ALLLEDGERENTRIES.LIST.LEDGERNAME','ALLLEDGERENTRIES.LIST.AMOUNT','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.DATE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.INSTRUMENTDATE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.NAME','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.TRANSACTIONTYPE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.CHEQUECROSSCOMMENT','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.UNIQUEREFERENCENUMBER','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.STATUS','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTMODE','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.BANKPARTYNAME','ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.AMOUNT','NARRATION']
df_raw_collect_soe = pd.DataFrame()
for xt in files_soe:
  path_ct = '/content/drive/My Drive/2024_25/SOE/'
  xml_file = path_ct+xt
  df_raw_f = xml_to_dataframe_vouchers(xml_file)[req_cols].reset_index(drop=True)
  df_raw_f_ref = pd.DataFrame(df_raw_f)
  df_raw_collect_soe = pd.concat([df_raw_collect_soe, df_raw_f_ref]).reset_index(drop=True)

df_raw_soe = df_raw_collect_soe.drop_duplicates(subset='GUID', keep='first').reset_index(drop=True)
df_raw_soe['date_column'] = pd.to_datetime(df_raw_soe['DATE'], format='%Y%m%d')
df_raw_soe = df_raw_soe.sort_values('date_column')

df_raw_collect_me = pd.DataFrame()
for xt in files_me:
  path_ct = '/content/drive/My Drive/2024_25/ME/'
  xml_file = path_ct+xt
  df_raw_f = xml_to_dataframe_vouchers(xml_file)[req_cols].reset_index(drop=True)
  df_raw_f_ref = pd.DataFrame(df_raw_f)
  df_raw_collect_me = pd.concat([df_raw_collect_me, df_raw_f_ref]).reset_index(drop=True)

df_raw_me = df_raw_collect_me.drop_duplicates(subset='GUID', keep='first').reset_index(drop=True)
df_raw_me['date_column'] = pd.to_datetime(df_raw_me['DATE'], format='%Y%m%d')
df_raw_me = df_raw_me.sort_values('date_column')

In [ ]:
def add_req_fields(df):
  df['party_name'] = df[['PARTYNAME', 'ALLLEDGERENTRIES.LIST.BANKALLOCATIONS.LIST.PAYMENTFAVOURING', 'ALLLEDGERENTRIES.LIST.LEDGERNAME']].bfill(axis=1).iloc[:, 0]
  df['tax_out_1'] = df[['LEDGERENTRIES.LIST.AMOUNT']].bfill(axis=1).iloc[:, 0]
  df['tax_out_2'] = df[['LEDGERENTRIES.LIST.VATEXPAMOUNT']].bfill(axis=1).iloc[:, 0]

  # df[''] = df[''].astype(float)
  df['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT'] = df['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT'].astype(float)
  df['tax_out_1'] = df['tax_out_1'].astype(float)
  df['tax_out_2'] = df['tax_out_2'].astype(float)

  df['amount_coalesce'] = df[['ALLINVENTORYENTRIES.LIST.BATCHALLOCATIONS.LIST.AMOUNT', 'ALLLEDGERENTRIES.LIST.AMOUNT']].bfill(axis=1).iloc[:, 0]
  df['amount_coalesce'] = df['amount_coalesce'].astype(float)
  df['tax_coalesce'] = df[['tax_out_1', 'tax_out_2']].sum(axis=1)
  df['total_coalesce'] = df[['tax_coalesce', 'amount_coalesce']].sum(axis=1)

  df = df[['party_name','total_coalesce','amount_coalesce','tax_coalesce','tax_out_1','tax_out_2']+req_cols]
  return df

In [ ]:
df_me = add_req_fields(df_raw_me)
df_soe = add_req_fields(df_raw_soe)

<ipython-input-13-bc721fa37c88>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['tax_out_1'] = df[['LEDGERENTRIES.LIST.AMOUNT']].bfill(axis=1).iloc[:, 0]
<ipython-input-13-bc721fa37c88>:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['tax_out_2'] = df[['LEDGERENTRIES.LIST.VATEXPAMOUNT']].bfill(axis=1).iloc[:, 0]
<ipython-input-13-bc721fa37c88>:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavi

In [ ]:

df_soe['total_coalesce'] = df_soe['total_coalesce'].astype('float')
df_summary_all_soe = pd.pivot_table(df_soe, values='total_coalesce', index='party_name', columns='VOUCHERTYPENAME', aggfunc='sum').reset_index()
df_summary_all_soe['balance'] = 0
df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='TIRUMALA MULTI TECHNOLOGIES'].index,'balance'] = 467646
df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='SAVITA AUTO INDUSTRIES (N)'].index,'balance'] = 195853
df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='PATIL-N-PATIL ENGINEERING	'].index,'balance'] = 55660
df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='NANDAMURI TECHNOS'].index,'balance'] = 35255
df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='V.S.AUTO TECH PRIVATE LIMITED'].index,'balance'] = 4699956
df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='A.B.ENGINEERING WORKS'].index,'balance'] = 11598

df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='AKAR INDUSTRY'].index,'balance'] = 317899

df_summary_all_soe.loc[df_summary_all_soe[df_summary_all_soe['party_name']=='SURYA AUTO INDUSTRIES'].index,'balance'] = 23165


df_summary_all_soe = df_summary_all_soe.fillna(0)
df_summary_all_soe['total_due']=df_summary_all_soe[['Contra', 'Credit Note', 'DELIVERY CHALLAN', 'Debit Note',
       'Journal', 'Payment', 'Purchase', 'Receipt', 'SALES GST', 'balance']].sum(axis=1)

df_me['total_coalesce'] = df_me['total_coalesce'].astype('float')
df_summary_all_me = pd.pivot_table(df_me, values='total_coalesce', index='party_name', columns='VOUCHERTYPENAME', aggfunc='sum').reset_index()
df_summary_all_me['balance'] = 0
df_summary_all_me = df_summary_all_me.fillna(0)
df_summary_all_me['total_due']=df_summary_all_me[[ 'Contra','Delivery Note','Debit Note','Sales Gst', 'Journal', 'Payment', 'Purchase', 'Receipt','Sales Gst','balance']].sum(axis=1)

<ipython-input-20-c45c9a62cf25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_soe['total_coalesce'] = df_soe['total_coalesce'].astype('float')
<ipython-input-20-c45c9a62cf25>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me['total_coalesce'] = df_me['total_coalesce'].astype('float')


In [ ]:
terms = ['SHREE SAI ENGINEERING', 'XYZ COMPANY'] # Function to match and replace terms def match_and_replace(transaction): for term in terms: if term.replace(" ", "").lower() in transaction.replace(" ", "").lower(): return term return transaction # Or return a default value if no match is found # Apply function to the column df['renamed_column'] = df['transaction_column'

In [ ]:
# df_bank_match
df_summary_all_bank_soe = pd.merge(df_summary_all_soe,df_bank_match_overview_soe,on=['party_name'],how='outer')
df_summary_all_bank_soe = df_summary_all_bank_soe.sort_values(by='SALES GST', key=lambda x: x.isna()).reset_index(drop=True).sort_values(by='SALES GST', ascending=False).reset_index(drop=True)

# df_bank_match
df_summary_all_bank_me = pd.merge(df_summary_all_me,df_bank_match_overview_me,on=['party_name'],how='outer')
df_summary_all_bank_me = df_summary_all_bank_me.sort_values(by='Sales Gst', key=lambda x: x.isna()).reset_index(drop=True).sort_values(by='Sales Gst', ascending=False).reset_index(drop=True)



In [ ]:
df_summary_all_bank_me.columns

Index(['party_name', 'Contra', 'Debit Note', 'Delivery Note', 'Journal',
       'Payment', 'Purchase', 'Receipt', 'Sales Gst', 'balance', 'total_due',
       'Debit', 'Credit'],
      dtype='object')

In [ ]:
import numpy as np
columns_to_round_soe = ['Contra', 'Credit Note', 'DELIVERY CHALLAN', 'Debit Note','Journal', 'Payment', 'Purchase', 'Receipt', 'SALES GST', 'balance','total_due', 'Debit', 'Credit']
df_summary_all_bank_soe[columns_to_round_soe] = df_summary_all_bank_soe[columns_to_round_soe].apply(np.ceil)

columns_to_round_me = ['Contra', 'Debit Note', 'Delivery Note', 'Journal','Payment', 'Purchase', 'Receipt', 'Sales Gst', 'balance', 'total_due','Debit', 'Credit']
df_summary_all_bank_me[columns_to_round_me] = df_summary_all_bank_me[columns_to_round_me].apply(np.ceil)

In [ ]:
df_summary_all_bank_soe.to_excel("/content/drive/My Drive/2024_25/daybook.xlsx",sheet_name='SOE_overview')
with pd.ExcelWriter("/content/drive/My Drive/2024_25/daybook.xlsx",mode='a') as writer:
  df_soe.to_excel(writer,sheet_name='SOE_raw_xml')

In [ ]:

with pd.ExcelWriter("/content/drive/My Drive/2024_25/daybook.xlsx",mode='a') as writer:
  df_summary_all_bank_me.to_excel(writer,sheet_name='ME_overview')
with pd.ExcelWriter("/content/drive/My Drive/2024_25/daybook.xlsx",mode='a') as writer:
  df_me.to_excel(writer,sheet_name='ME_raw_xml')

# for xrow in df['VOUCHERTYPENAME'].unique().tolist():
#   with pd.ExcelWriter("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",mode='a') as writer:
#     df[df['VOUCHERTYPENAME']==xrow].reset_index(drop=True).to_excel(writer,sheet_name=xrow)


In [ ]:
# import os
# from datetime import datetime
# path = '/content/drive/My Drive/2024_25'  # Replace with your desired path
# files = os.listdir(path)
# old_file_path  = '/content/drive/My Drive/2024_25/SOEDayBook.xlsx'
# today_date = datetime.today().strftime('%Y-%m-%d')
# if 'SOEDayBook.xlsx'  in files:
#     # Create the new file name by appending today's date
#   file_name, file_extension = os.path.splitext(old_file_path)
#   new_file_path = f"{file_name}_{today_date}{file_extension}"

#   # Rename the file
#   os.rename(old_file_path, new_file_path)
#   if not os.path.exists(old_file_path) and os.path.exists(new_file_path):
#       print(f"File renamed and saved as: {new_file_path}")
#       df.to_excel("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",sheet_name='master_sheet')

#       for xrow in df['VOUCHERTYPENAME'].unique().tolist():
#         with pd.ExcelWriter("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",mode='a') as writer:
#           df[df['VOUCHERTYPENAME']==xrow].reset_index(drop=True).to_excel(writer,sheet_name=xrow)
#   else:
#       print(f"Error renaming file: {old_file_path}")

In [ ]:
# import pandas as pd
# df['DATE'] = pd.to_datetime(df['DATE'])

# df_sales = df[df['VOUCHERTYPENAME']=='SALES GST'].reset_index(drop=True)
# # Calculate new dates
# df_sales['DATE_plus_45'] = df_sales['DATE'] + pd.DateOffset(days=45)
# df_sales['DATE_plus_60'] = df_sales['DATE'] + pd.DateOffset(days=60)
# df_sales['DATE_plus_90'] = df_sales['DATE'] + pd.DateOffset(days=90)
# df_sales['Week_Number'] = df_sales['DATE_plus_45'].dt.isocalendar().week
# df_sales['ALLINVENTORYENTRIES.LIST.AMOUNT'] = df_sales['ALLINVENTORYENTRIES.LIST.AMOUNT'].astype(float)
# df_sales = df_sales.rename(columns={'ALLINVENTORYENTRIES.LIST.AMOUNT': 'sales_amount'})
# df_sales_g = df_sales.groupby(['PARTYLEDGERNAME','Week_Number'])['sales_amount'].sum().reset_index()

# # Add a column for the payment due date (45 days after the sale date)
# df_sales['due_date'] = df_sales['DATE'] + pd.Timedelta(days=45)

# # Filter the DataFrame to include only pending payments (due date is in the future)
# pending_payments = df_sales[df_sales['due_date'] > pd.Timestamp.now()]

# # Group by customer name and week of the year, and sum the amounts
# pending_payments_per_week = pending_payments.groupby(['PARTYLEDGERNAME'])['sales_amount'].sum().reset_index()

# pending_payments_per_week.columns = ['PARTYLEDGERNAME', 'due_amount_'+str(pd.Timestamp.now().date())]
# outstanding_payments_g = df_sales.groupby(['PARTYLEDGERNAME'])['sales_amount'].sum().reset_index()

# outstanding_payments_g.columns = ['PARTYLEDGERNAME', 'outstanding_amount']
# df_receipt = df[df['VOUCHERTYPENAME']=='Receipt'].reset_index(drop=True)
# df_receipt['ALLLEDGERENTRIES.LIST.AMOUNT'] = df_receipt['ALLLEDGERENTRIES.LIST.AMOUNT'].astype(float)
# df_receipt = df_receipt.rename(columns={'ALLLEDGERENTRIES.LIST.AMOUNT': 'receipt_amount'})
# df_receipt_g = df_receipt.groupby(['PARTYLEDGERNAME'])['receipt_amount'].sum().reset_index()
# df_due_45_days = pd.merge(df_receipt_g,outstanding_payments_g,on=['PARTYLEDGERNAME'],how='right')
# df_due_45_days = pd.merge(df_due_45_days,pending_payments_per_week,on=['PARTYLEDGERNAME'],how='left')
# df_due_45_days['closing_balance_on_31_march'] = ''
# with pd.ExcelWriter("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",mode='a') as writer:
#   df_due_45_days.to_excel(writer,sheet_name='Due_45_days')


/usr/local/lib/python3.10/dist-packages/openpyxl/packaging/relationship.py:118: UserWarning: xl/pivotCache/_rels/pivotCacheDefinition1.xml.rels contains invalid dependency definitions
  warn(msg)


ValueError: Sheet 'Due_45_days' already exists and if_sheet_exists is set to 'error'.